In [78]:
import pathlib
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint

namecheckpoint = 'TreinosTF/ResNet50V2/v40/model.{epoch:02d}-{val_accuracy:.2f}.h5' 
model_checkpoint_callback = ModelCheckpoint(namecheckpoint,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    # change output folder
 )
# substitui sua base aqui
data_dir = pathlib.Path("./dataset brasileiro/TREINO")
 
# Define the classes from the data_dir
classes = pathlib.Path(data_dir).glob('*')
classes = [x for x in classes if x.is_dir()]
classes = sorted([x.stem for x in classes])
print(classes)

# Define batch size
batch_size = 32
img_height = 224
img_width = 224

epochs = 10
# Define the data directory

# Create training dataset
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Create validation dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Load the pre-trained MobileNetV2 model
pre_trained_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

total_layers = len(pre_trained_model.layers)
num_layers_to_freeze = int(0.4 * total_layers)
for layer in pre_trained_model.layers[:num_layers_to_freeze]:
    layer.trainable = False

# Add custom classification head
x = layers.Flatten()(pre_trained_model.output)
predictions = layers.Dense(len(classes), activation='softmax')(x)

model3 = Model(inputs=pre_trained_model.input, outputs=predictions)

# Compile the model
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model using the training dataset and validation dataset
history = model3.fit(
    dataset, 
    validation_data=validation_dataset, 
    epochs=epochs,
    callbacks=[model_checkpoint_callback]  
    )

['AMERICAN KESTREL', 'AMERICAN REDSTART', 'AMETHYST WOODSTAR', 'ANHINGA', 'BAR-TAILED GODWIT', 'BARN OWL', 'BARN SWALLOW', 'BEARDED BELLBIRD', 'BLACK NECKED STILT', 'BLACK SKIMMER', 'BLACK VULTURE', 'BLACKBURNIAM WARBLER', 'BLONDE CRESTED WOODPECKER', 'BOBOLINK', 'CAMPO FLICKER', 'CAPPED HERON', 'CAPUCHINBIRD', 'CINNAMON ATTILA', 'CINNAMON TEAL', 'CRANE HAWK', 'CREAM COLORED WOODPECKER', 'EASTERN MEADOWLARK', 'GREAT JACAMAR', 'GREAT KISKADEE', 'GREAT POTOO', 'GREAT XENOPS', 'GREY PLOVER', 'HARPY EAGLE', 'HORNED SUNGEM', 'HOUSE SPARROW', 'IVORY BILLED ARACARI', 'KING VULTURE', 'LAUGHING GULL', 'LIMPKIN', 'MANGROVE CUCKOO', 'ORNATE HAWK EAGLE', 'OSPREY', 'PEREGRINE FALCON', 'POMARINE JAEGER', 'PURPLE GALLINULE', 'PURPLE MARTIN', 'RED BILLED TROPICBIRD', 'RED KNOT', 'ROCK DOVE', 'ROSEATE SPOONBILL', 'RUFUOS MOTMOT', 'SAND MARTIN', 'SCARLET MACAW', 'SHORT BILLED DOWITCHER', 'SNOWY EGRET', 'SNOWY SHEATHBILL', 'SPANGLED COTINGA', 'SQUACCO HERON', 'STRIPED OWL', 'STRIPPED MANAKIN', 'SUNBITTER

In [79]:
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
import scipy
from keras.models import load_model

#model3 = load_model('./TreinosTF/MobileNetV2/v3/model.06-0.80.h5')

# Defina os valores de altura e largura da imagem aqui
img_height = 224  # Substitua pelo valor correto
img_width = 224   # Substitua pelo valor correto

# Defina o tamanho do lote (batch_size) aqui
batch_size = 32  # Substitua pelo valor desejado

test_datagen = ImageDataGenerator()  # Rescale pixel values to [0, 1]
test_data_dir = "./dataset brasileiro/TEST"


# Create a test generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),  # Adjust the target size to match your model's input size
    batch_size=batch_size,  # Batch size for generating predictions
    class_mode='categorical',  # Change this to 'binary' i  f you have a binary classification problem
    shuffle=False  # Set to False to maintain the order of images
)

# Get the class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Calculate the number of images in the dataset
num_images = len(test_generator.filenames)

# Initialize empty lists to store images and true labels
all_images = []
all_true_labels = []

# Loop through the generator to accumulate all images and true labels
for i in range(num_images // batch_size):
    images, true_labels = next(test_generator)
    all_images.append(images)
    all_true_labels.append(true_labels)

# Concatenate the accumulated images and true labels
all_images = np.concatenate(all_images)
all_true_labels = np.concatenate(all_true_labels)

# Make predictions using the model on all images
predictions = model3.predict(all_images)

# Now, you have predictions for all images in the test dataset
print("Total number of images:", len(all_images))
print(class_labels)

Found 310 images belonging to 62 classes.
9/9 [==============================] - 1s 45ms/step
Total number of images: 288
['AMERICAN KESTREL', 'AMERICAN REDSTART', 'AMETHYST WOODSTAR', 'ANHINGA', 'BAR-TAILED GODWIT', 'BARN OWL', 'BARN SWALLOW', 'BEARDED BELLBIRD', 'BLACK NECKED STILT', 'BLACK SKIMMER', 'BLACK VULTURE', 'BLACKBURNIAM WARBLER', 'BLONDE CRESTED WOODPECKER', 'BOBOLINK', 'CAMPO FLICKER', 'CAPPED HERON', 'CAPUCHINBIRD', 'CINNAMON ATTILA', 'CINNAMON TEAL', 'CRANE HAWK', 'CREAM COLORED WOODPECKER', 'EASTERN MEADOWLARK', 'GREAT JACAMAR', 'GREAT KISKADEE', 'GREAT POTOO', 'GREAT XENOPS', 'GREY PLOVER', 'HARPY EAGLE', 'HORNED SUNGEM', 'HOUSE SPARROW', 'IVORY BILLED ARACARI', 'KING VULTURE', 'LAUGHING GULL', 'LIMPKIN', 'MANGROVE CUCKOO', 'ORNATE HAWK EAGLE', 'OSPREY', 'PEREGRINE FALCON', 'POMARINE JAEGER', 'PURPLE GALLINULE', 'PURPLE MARTIN', 'RED BILLED TROPICBIRD', 'RED KNOT', 'ROCK DOVE', 'ROSEATE SPOONBILL', 'RUFUOS MOTMOT', 'SAND MARTIN', 'SCARLET MACAW', 'SHORT BILLED DOWITCH

In [83]:
print(predicted_labels)

[ 7 32 54  0 33 17  1  6 29 39  2 13 44 28 54  3  3  3  3  3  4  4  4 48
  4  5 49 49 13  5  6  6  6  6  6  7 36  7  0  7 33  6  1  8 62 62 62 49
 38 21 29 33  7 10 55 21 49  6 54 11 12 12 12 12 12 21 13 13 13 58 14 33
 19  4 29 49 15 17 49 49 22 20 16 20 59 20 17 17 17 17 18 54 62 46 18 41
 39 11  6 39 20  3 20 54 20 29 42 21 21 21 22 22 22 22 22 23 23 31 54 23
 24 53 53 24 24 47 20 44 20 42  4 26  4 26 26 31 62 41 22 53 28 28 28 28
 28  6 54 26 29 53 56 22 51 30 30  2 31 47 59  6 32 32 38 32 32 62 33 33
 33  4 62 20 45 45 34 41 24 62  4 33 51 53 57 36  0 33 43 57 33 37 37 62
 32 57 26 39 39 39 56 39  6  6  0 40 40 41 62 31 62 41 42 42 24 42  4 38
  7 33  4 33 50 50 49 50 58 45 16 54 17 45 59 37 56 56 56 47 47 25 47 47
  4 42 48 48  4 49 62 32 49 49 62 50 50 49 49 51 59 51 51 31 39 25 52 44
 52 49  5 24 53 53 54 39 39 54 20  4 33 62 26 62 56 54 58 56 32 57 36 43]


In [81]:
import numpy as np

# Inicialize as listas vazias para armazenar os rótulos previstos e verdadeiros
predicted_labels = []
labels_true = []

# Converta as probabilidades previstas em rótulos de classe
for pred in predictions:
    predicted_label = class_labels[np.argmax(pred)]
    predicted_labels.append(predicted_label)

# Converta os rótulos verdadeiros one-hot em rótulos de classe
for true in true_labels:
    true_label = class_labels[np.argmax(true)]
    labels_true.append(true_label)

# Calcule a precisão
accuracy = np.mean(np.equal(predicted_labels, labels_true))
accuracy_percentage = accuracy * 100

print("Accuracy:", accuracy_percentage, "%")


IndexError: list index out of range

In [85]:
# Converta as probabilidades previstas em rótulos de classe (índices das classes)
predicted_labels = np.argmax(predictions, axis=1)

# Converta as etiquetas verdadeiras one-hot em rótulos de classe (índices das classes)
true_labels = np.argmax(all_true_labels, axis=1)

# Calcule a acurácia comparando os rótulos previstos com os rótulos verdadeiros
accuracy = np.mean(np.equal(predicted_labels, true_labels))
accuracy_percentage = accuracy * 100

print("Accuracy:", accuracy_percentage, "%")


Accuracy: 40.625 %
